In [19]:
from wigner_functions import *
import h5py
from scipy.sparse import *
import zarr

In [20]:
import time

In [40]:
lmax=1e4
wlmax=5e2
lmax=np.int(lmax)
wlmax=np.int(wlmax)

In [41]:
ncpu=8
l_step=10 #not used with dask
w_l=np.arange(wlmax)
l=np.arange(lmax)

In [42]:
import dask
import dask.array as da
from dask import delayed

from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
LC=LocalCluster(n_workers=1,processes=False,memory_limit='50gb',threads_per_worker=ncpu,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(LC)

In [43]:
def wigner_3j_asym(j_1,j_2,j_3,m_1,m_2,m_3): #assume j1,j2>>j3.... wiki
    sj=(j_1+j_2+1)
    th=np.arccos((m_1-m_2)/sj)
    wd=wigner_d(m_3,j_2-j_1,np.atleast_1d(th),j_3)[0,0]
    print(wd)
    return ((-1)**(j_2+m_2))*wd/np.sqrt(sj)

In [44]:
def wigner_3j_asym_H(j_1,j_2,j_3,m_1,m_2,m_3): #j1+j2+j3>>1, m1=m2=m3=0.. Hivon
    wj=2*(j1*j2)**2
    wj+=2*(j2*j3)**2
    wj+=2*(j1*j3)**2
    wj-=j1**4+j2**4+j3**4
    wj=wj**(-0.5)
    wj*=2./np.pi
    return np.sqrt(wj)*(-1)**((j_1+j_2+j_3)/2)

def wigner_3j_asym_H2(j_1,j_2,j_3,m_1,m_2,m_3): #j1+j2+j3>>1, m1=m2=m3=0.. Hivon
    J=j_1+j_2+j_3
    logwj=log_factorial(J/2)
    logwj-=log_factorial(J/2-j_1)
    logwj-=log_factorial(J/2-j_2)
    logwj-=log_factorial(J/2-j_3)
    logwj-=0.5*log_factorial(J+1)
    logwj+=0.5*log_factorial(J-2*j_1)
    logwj+=0.5*log_factorial(J-2*j_2)
    logwj+=0.5*log_factorial(J-2*j_3)
    return (-1)**(J/2)*np.exp(logwj)

In [45]:
def wig3j_map(m1,m2,m3,j1,j2,j3,asym_fact=np.inf):
#     n1=len(l)
#     n2=len(l)
#     n3=len(j3)

#     c=np.array(list(Comb(l,l,j3)))
# #     print(c.shape)
#     j_max=np.amax(l.max()+l.max()+l.max()+1)
#     _calc_factlist(j_max)
#     d_mat=client.gather(client.map(partial(wigner_3j_3, m_1, m_2, m_3),c))
#     d_mat=np.array(d_mat).reshape(n1,n2,n3)
    
    return Wigner3j_parallel( m1, m2, m3,np.atleast_1d(j1), np.atleast_1d(j2) ,np.atleast_1d(j3),ncpu=1,
                             asym_fact=asym_fact)[:,:,0]


# dst=client.map(wig3j_map,w_l)

In [46]:
def Wigner3j_parallel2( m_1, m_2, m_3,j_1, j_2, j_3,ncpu=None,asym_fact=np.inf):
    if ncpu is None:
        ncpu=cpu_count()-2
    p=Pool(ncpu)

    j_max=np.amax(j_1.max()+j_2.max()+j_3.max()+1)
#     _calc_factlist(j_max)

    n1=len(j_1)
    n2=len(j_2)
    n3=len(j_3)

    c=np.array(np.meshgrid(j_1,j_2,j_3,indexing='ij')).T.reshape(-1,3) #only needed to put cuts below. Otherwise Comb is better

    x=c[:,0]+c[:,1]-c[:,2]>=0
    x*=c[:,0]-c[:,1]+c[:,2]>=0
    x*=-c[:,0]+c[:,1]+c[:,2]>=0
    
    x*=abs(m_1) <= c[:,0]
    x*=abs(m_2) <= c[:,1]
    x*=abs(m_3) <= c[:,2]
    
    if m_1==0 and m_2==0 and m_3==0:
        x*=(c[:,0]+c[:,1]+c[:,2])%2==0

    c=c[x]
    
    
    d_mat=p.map(partial(wigner_3j_3,asym_fact, m_1, m_2, m_3),c,chunksize=100)
    p.close()
    
    dd=np.zeros((n1,n2,n3))
    indx1=np.searchsorted(j_1,c[:,0])
    indx2=np.searchsorted(j_2,c[:,1])
    indx3=np.searchsorted(j_3,c[:,2])
    dd[indx1,indx2,indx3]=d_mat
    
    return dd

In [47]:
asym_fact=50

In [48]:
# if lmax>500:
#     fname='temp/dask_wig3j_big_{i}.zarr'
# else:
#     fname='temp/dask_wig3j_test_{i}.zarr'
fname='temp/dask_wig3j_l{lmax}_w{wlmax}_{i}.zarr'
if asym_fact is not np.inf:
    fname='temp/dask_wig3j_l{lmax}_w{wlmax}_{i}_asym'+str(asym_fact)+'.zarr'

In [49]:
step=np.int(min(5e3,lmax/10))
lb=np.sort(np.unique(np.append([0,100,lmax],np.arange(100,lmax,step))))
lb,lb.shape

(array([    0,   100,  1100,  2100,  3100,  4100,  5100,  6100,  7100,
         8100,  9100, 10000]), (12,))

In [50]:
12*12*500

72000

In [51]:
m1=0
m2=0
m3=0
# arrs=[da.from_delayed(delayed(wig3j_map)(m1,m2,m3,l,l,np.atleast_1d(j3),asym_fact),shape=(lmax,lmax),dtype='float32') for j3 in w_l]

arrs=[da.hstack([da.vstack([da.from_delayed(delayed(wig3j_map)(m1,m2,m3,l[lb[i]:lb[i+1]],l[lb[k]:lb[k+1]],
                                                               np.atleast_1d(j3),asym_fact),
                    shape=(lb[i+1]-lb[i],lb[k+1]-lb[k]),dtype='float32') 
                    for i in np.arange(len(lb)-1)]) 
                     for k in np.arange(len(lb)-1)])
                      for j3 in w_l]
arrs2=da.stack(arrs)
arrs2=arrs2.rechunk(chunks=(1,1000,1000))

%time arrs2.to_zarr(fname.format(i=0,lmax=lmax,wlmax=wlmax), overwrite=True)

CPU times: user 1h 35min 18s, sys: 3min 16s, total: 1h 38min 35s
Wall time: 16min 28s


In [15]:
m1=-2
m2=2
m3=0

arrs=[da.hstack([da.vstack([da.from_delayed(delayed(wig3j_map)(m1,m2,m3,l[lb[i]:lb[i+1]],l[lb[k]:lb[k+1]],
                                                               np.atleast_1d(j3),asym_fact),
                    shape=(lb[i+1]-lb[i],lb[k+1]-lb[k]),dtype='float32') 
                    for i in np.arange(len(lb)-1)]) 
                     for k in np.arange(len(lb)-1)])
                      for j3 in w_l]
arrs2=da.stack(arrs)
arrs2=arrs2.rechunk(chunks=(1,1000,1000))
%time arrs2.to_zarr(fname.format(i=2,lmax=lmax,wlmax=wlmax),overwrite=True)

[213] 99 1099 done wig time,size:  65.9208927154541 29988 [ 99 312 213]
[213] 1099 99 done wig time,size:  66.396066904068 29988 [312  99 213]
[90] 99 1099 done wig time,size:  18.00678014755249 12285 [ 99 189  90]
[90] 1099 99 done wig time,size:  17.995177507400513 12285 [189  99  90]
[90] 99 99 done wig time,size:  20.515414476394653 17421 [99 99 90]
[355] 1099 99 done wig time,size:  110.60238456726074 29988 [454  99 355]
[355] 99 1099 done wig time,size:  111.37614011764526 29988 [ 99 454 355]
[7] 1099 99 done wig time,size:  0.10157370567321777 84 [106  99   7]
[7] 99 1099 done wig time,size:  0.13439297676086426 84 [ 99 106   7]
[7] 99 99 done wig time,size:  2.7628326416015625 4224 [99 99  7]
[97] 99 1099 done wig time,size:  22.363945245742798 14259 [ 99 196  97]
[97] 1099 99 done wig time,size:  22.20711636543274 14259 [196  99  97]
[432] 99 1099 done wig time,size:  143.58293509483337 29988 [ 99 531 432]
[432] 1099 99 done wig time,size:  143.73137950897217 29988 [531  99 43

[441] 1099 99 done wig time,size:  148.08226704597473 29988 [540  99 441]
[389] 99 1099 done wig time,size:  123.26266670227051 29988 [ 99 488 389]
[187] 99 99 done wig time,size:  0.22899675369262695 234 [ 99  99 187]
[389] 1099 99 done wig time,size:  123.77465748786926 29988 [488  99 389]
[89] 1099 99 done wig time,size:  17.812363147735596 12015 [188  99  89]
[89] 99 1099 done wig time,size:  17.93657875061035 12015 [ 99 188  89]
[8] 1099 99 done wig time,size:  0.12123632431030273 108 [107  99   8]
[8] 99 1099 done wig time,size:  0.12632036209106445 108 [ 99 107   8]
[8] 99 99 done wig time,size:  2.1795027256011963 4752 [99 99  8]
[187] 99 1099 done wig time,size:  60.07219171524048 29754 [ 99 286 187]
[89] 99 99 done wig time,size:  21.40042471885681 17631 [99 99 89]
[6] 99 1099 done wig time,size:  0.10593843460083008 63 [ 99 105   6]
[6] 1099 99 done wig time,size:  0.08304619789123535 63 [105  99   6]
[6] 99 99 done wig time,size:  1.6874675750732422 3687 [99 99  6]
[260] 99

[399] 99 1099 done wig time,size:  127.35112500190735 29988 [ 99 498 399]
[399] 1099 99 done wig time,size:  129.12049269676208 29988 [498  99 399]
[351] 99 1099 done wig time,size:  108.27347946166992 29988 [ 99 450 351]
[351] 1099 99 done wig time,size:  108.65394949913025 29988 [450  99 351]
[270] 99 1099 done wig time,size:  79.43848705291748 29988 [ 99 369 270]
[270] 1099 99 done wig time,size:  80.29583382606506 29988 [369  99 270]
[489] 99 1099 done wig time,size:  171.58144974708557 29988 [ 99 588 489]
[489] 1099 99 done wig time,size:  172.09440326690674 29988 [588  99 489]
[225] 99 1099 done wig time,size:  68.02793622016907 29988 [ 99 324 225]
[225] 1099 99 done wig time,size:  68.71818947792053 29988 [324  99 225]
[354] 99 1099 done wig time,size:  110.23067474365234 29988 [ 99 453 354]
[354] 1099 99 done wig time,size:  110.69294118881226 29988 [453  99 354]
[323] 99 1099 done wig time,size:  97.28912043571472 29988 [ 99 422 323]
[323] 1099 99 done wig time,size:  97.49786

[144] 99 1099 done wig time,size:  45.315181493759155 25368 [ 99 243 144]
[44] 99 1099 done wig time,size:  3.2177248001098633 2970 [ 99 143  44]
[44] 1099 99 done wig time,size:  3.452488422393799 2970 [143  99  44]
[44] 99 99 done wig time,size:  16.308421850204468 17766 [99 99 44]
[259] 99 1099 done wig time,size:  77.23541641235352 29988 [ 99 358 259]
[259] 1099 99 done wig time,size:  77.89746236801147 29988 [358  99 259]
[396] 99 1099 done wig time,size:  127.1022617816925 29988 [ 99 495 396]
[396] 1099 99 done wig time,size:  127.63365316390991 29988 [495  99 396]
[9] 1099 99 done wig time,size:  0.18004250526428223 135 [108  99   9]
[9] 99 1099 done wig time,size:  0.2740790843963623 135 [ 99 108   9]
[9] 99 99 done wig time,size:  2.5234553813934326 5271 [99 99  9]
[107] 99 1099 done wig time,size:  26.997697830200195 17154 [ 99 206 107]
[107] 1099 99 done wig time,size:  27.302861213684082 17154 [206  99 107]
[107] 99 99 done wig time,size:  15.107650995254517 12834 [ 99  99 

[81] 99 1099 done wig time,size:  15.03759765625 9963 [ 99 180  81]
[81] 1099 99 done wig time,size:  15.821100234985352 9963 [180  99  81]
[179] 99 99 done wig time,size:  0.4938483238220215 630 [ 99  99 179]
[81] 99 99 done wig time,size:  22.382704496383667 18987 [99 99 81]
[474] 99 1099 done wig time,size:  164.72084617614746 29988 [ 99 573 474]
[474] 1099 99 done wig time,size:  165.57504391670227 29988 [573  99 474]
[179] 99 1099 done wig time,size:  57.40658473968506 29358 [ 99 278 179]
[52] 1099 99 done wig time,size:  5.989909648895264 4134 [151  99  52]
[179] 1099 99 done wig time,size:  58.38579797744751 29358 [278  99 179]
[52] 99 1099 done wig time,size:  5.5746331214904785 4134 [ 99 151  52]
[421] 1099 99 done wig time,size:  138.7694399356842 29988 [520  99 421]
[421] 99 1099 done wig time,size:  139.36393356323242 29988 [ 99 520 421]
[52] 99 99 done wig time,size:  19.40319013595581 19074 [99 99 52]
[369] 99 1099 done wig time,size:  116.79187202453613 29988 [ 99 468 36

...

[46] 99 1099 done wig time,size:  3.5855400562286377 3243 [ 99 145  46]
[46] 1099 99 done wig time,size:  3.6053659915924072 3243 [145  99  46]
[173] 1099 99 done wig time,size:  56.61601805686951 28935 [272  99 173]
[173] 99 1099 done wig time,size:  57.10290026664734 28935 [ 99 272 173]
[112] 99 99 done wig time,size:  14.356804370880127 11484 [ 99  99 112]
[46] 99 99 done wig time,size:  17.75682806968689 18147 [99 99 46]
[112] 99 1099 done wig time,size:  30.491844415664673 18504 [ 99 211 112]
[112] 1099 99 done wig time,size:  30.065762281417847 18504 [211  99 112]
[262] 99 1099 done wig time,size:  78.93213129043579 29988 [ 99 361 262]
[183] 99 99 done wig time,size:  0.35436415672302246 408 [ 99  99 183]
[262] 1099 99 done wig time,size:  79.5796103477478 29988 [361  99 262]
[236] 99 1099 done wig time,size:  71.74169754981995 29988 [ 99 335 236]
[236] 1099 99 done wig time,size:  72.47077894210815 29988 [335  99 236]
[183] 99 1099 done wig time,size:  59.035407066345215 29580 [

[42] 99 1099 done wig time,size:  4.042399883270264 2709 [ 99 141  42]
[42] 1099 99 done wig time,size:  4.203089237213135 2709 [141  99  42]
[87] 99 99 done wig time,size:  22.276345252990723 18024 [99 99 87]
[42] 99 99 done wig time,size:  16.056549549102783 17349 [99 99 42]
[283] 99 1099 done wig time,size:  84.58258438110352 29988 [ 99 382 283]
[283] 1099 99 done wig time,size:  86.52193641662598 29988 [382  99 283]
[308] 99 1099 done wig time,size:  93.60179686546326 29988 [ 99 407 308]
[308] 1099 99 done wig time,size:  93.9427125453949 29988 [407  99 308]
[232] 1099 99 done wig time,size:  70.43603730201721 29988 [331  99 232]
[232] 99 1099 done wig time,size:  71.35492658615112 29988 [ 99 331 232]
[13] 99 1099 done wig time,size:  0.2952864170074463 273 [ 99 112  13]
[13] 1099 99 done wig time,size:  0.3758971691131592 273 [112  99  13]
[219] 99 1099 done wig time,size:  67.40738010406494 29988 [ 99 318 219]
[219] 1099 99 done wig time,size:  68.50281310081482 29988 [318  99 21

[108] 99 99 done wig time,size:  15.120166301727295 12558 [ 99  99 108]
[29] 1099 99 done wig time,size:  1.3467683792114258 1305 [128  99  29]
[108] 1099 99 done wig time,size:  28.161349534988403 17430 [207  99 108]
[29] 99 99 done wig time,size:  10.518489122390747 13761 [99 99 29]
[55] 99 99 done wig time,size:  19.643166303634644 19416 [99 99 55]
[118] 99 99 done wig time,size:  10.890812635421753 9963 [ 99  99 118]
[92] 99 1099 done wig time,size:  18.977358102798462 12834 [ 99 191  92]
[118] 99 1099 done wig time,size:  32.57116198539734 20025 [ 99 217 118]
[92] 1099 99 done wig time,size:  18.602786779403687 12834 [191  99  92]
[118] 1099 99 done wig time,size:  33.177427530288696 20025 [217  99 118]
[181] 99 99 done wig time,size:  0.38666868209838867 513 [ 99  99 181]
[92] 99 99 done wig time,size:  20.337130784988403 16974 [99 99 92]
[438] 1099 99 done wig time,size:  146.5338807106018 29988 [537  99 438]
[438] 99 1099 done wig time,size:  147.7857038974762 29988 [ 99 537 43

[384] 99 1099 done wig time,size:  122.15185141563416 29988 [ 99 483 384]
[384] 1099 99 done wig time,size:  123.29527187347412 29988 [483  99 384]
[330] 99 1099 done wig time,size:  100.61658477783203 29988 [ 99 429 330]
[455] 99 1099 done wig time,size:  155.0691499710083 29988 [ 99 554 455]
[330] 1099 99 done wig time,size:  101.27943515777588 29988 [429  99 330]
[20] 99 1099 done wig time,size:  0.7300136089324951 630 [ 99 119  20]
[20] 1099 99 done wig time,size:  0.7065882682800293 630 [119  99  20]
[455] 1099 99 done wig time,size:  155.7389087677002 29988 [554  99 455]
[30] 99 1099 done wig time,size:  1.4575142860412598 1395 [ 99 129  30]
[30] 1099 99 done wig time,size:  1.487736701965332 1395 [129  99  30]
[20] 99 99 done wig time,size:  8.456028938293457 10386 [99 99 20]
[30] 99 99 done wig time,size:  12.368030548095703 14091 [99 99 30]
[48] 99 1099 done wig time,size:  3.9691195487976074 3528 [ 99 147  48]
[48] 1099 99 done wig time,size:  3.885620355606079 3528 [147  99 

[220] 1099 99 done wig time,size:  69.26849317550659 29988 [319  99 220]
[131] 1099 99 done wig time,size:  39.93683648109436 22950 [230  99 131]
[131] 99 1099 done wig time,size:  40.222820520401 22950 [ 99 230 131]
[185] 99 1099 done wig time,size:  59.79934358596802 29673 [ 99 284 185]
[185] 1099 99 done wig time,size:  60.43157386779785 29673 [284  99 185]
[229] 99 1099 done wig time,size:  70.5528039932251 29988 [ 99 328 229]
[229] 1099 99 done wig time,size:  70.82150554656982 29988 [328  99 229]
[239] 99 1099 done wig time,size:  72.53132462501526 29988 [ 99 338 239]
[239] 1099 99 done wig time,size:  73.49147057533264 29988 [338  99 239]
[292] 99 1099 done wig time,size:  88.01976704597473 29988 [ 99 391 292]
[292] 1099 99 done wig time,size:  88.98284578323364 29988 [391  99 292]
[267] 1099 99 done wig time,size:  80.54166984558105 29988 [366  99 267]
[267] 99 1099 done wig time,size:  80.74402332305908 29988 [ 99 366 267]
[114] 99 1099 done wig time,size:  32.14038968086243 1

In [16]:
arrs2

dask.array<rechunk-merge, shape=(500, 5000, 5000), dtype=float32, chunksize=(1, 1000, 1000)>

In [17]:
z1=zarr.open('./temp/dask_wig3j_l100_w30_0_asym50.zarr/')
z2=np.array(zarr.open('./temp/dask_wig3j_l100_w30_0.zarr/'))

In [18]:
z1.reshape(1,3000,100)

AttributeError: 'Array' object has no attribute 'reshape'

In [ ]:
crash

In [ ]:
zaa_in=zarr.open('temp/dask_test0.zarr')

In [ ]:
np.einsum('i,ijk->jk',w_l,zaa_in)

In [ ]:
zaa_in.oindex[np.int32(w_l[:5]), [1, 3],[1,5]]

In [ ]:
zaa_in

In [ ]:
aa2 = da.from_array(zaa_in, chunks=zaa_in.chunks)

In [ ]:
aa2

In [ ]:
m_1=0
m_2=0
m_3=0
fname='temp/wigner_test_big.h5'

with h5py.File(fname,'w') as f:
    dst = f.create_dataset("0", shape=(lmax,lmax,wlmax),
                           dtype=np.float32)
    lm=0
    while lm<lmax:
        l_t=np.arange(lm,lm+l_step)
        lm2=0
        while lm2<lmax:
            if lm2>lm+l_step+wlmax or lm>lm2+l_step+wlmax:
                print(lm2,lm)
                lm2+=l_step
                continue
            l_t2=np.arange(lm2,lm2+l_step)
            dst[lm:lm+l_step,lm2:lm2+l_step,:]=Wigner3j_parallel( m_1, m_2, m_3,l_t, l_t2 , w_l,ncpu=ncpu)
            
            lm2+=l_step
            
        lm+=l_step

In [ ]:
f.close()

In [ ]:
m_1=2
m_2=-2
m_3=0

with h5py.File(fname,'a') as f:
    dst = f.create_dataset("2", shape=(lmax,lmax,wlmax),
                           dtype=np.float32)
    lm=0
    while lm<lmax:
        l_t=np.arange(lm,lm+l_step)
        lm2=0
        while lm2<lmax:
            if lm2>lm+l_step+wlmax or lm>lm2+l_step+wlmax:
                print(lm2,lm)
                lm2+=l_step
                continue
            l_t2=np.arange(lm2,lm2+l_step)
            dst[lm:lm+l_step,lm2:lm2+l_step,:]=Wigner3j_parallel( m_1, m_2, m_3,l_t, l_t2 , w_l,ncpu=ncpu)
            lm2+=l_step
        lm+=l_step

In [ ]:
f.close()

In [ ]:
fname='temp/wigner_test.h5'

In [ ]:
f = h5py.File(fname, 'r')


In [ ]:
d=f["0"]

In [ ]:
d.shape

In [ ]:
w=np.ones_like(w_l)

In [ ]:
M=np.zeros((500,500))
M2=np.zeros((500,500))
M[:,:]=np.dot(d,w)

In [ ]:
x=d[w_l[3:5],:,:]#

In [ ]:
x[:,np.int32(w_l[:5]),:]

In [ ]:
lm=0
while lm<lmax:
    M2[lm:lm+l_step,:]=np.dot(d[lm:lm+l_step,:,:],w)
    lm+=l_step

In [ ]:
np.all(M2==M)

In [ ]:
f.close()

In [ ]:
coo_matrix((100,100))